In [38]:
import requests
import json
import pymysql
from newspaper import Article
from bs4 import BeautifulSoup

In [42]:
# MariaDB Connect
db = pymysql.connect(host='localhost', port=3306, user='root', passwd='root', db='trackle', charset='utf8',autocommit=True)
cursor = db.cursor()

# News 크롤링
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows; U; MSIE 9.0; WIndows NT 9.0; ko-KR))',
}

req = requests.get("https://search.naver.com/search.naver?where=news&sm=tab_jum&query=", headers=headers)

html = req.text
soup = BeautifulSoup(html, "lxml")

lst_realtime_src = soup.find('ol', {"class":"lst_realtime_srch _tab_area"})
lis = lst_realtime_src.findAll('li')

articles = {"article":[]}

for li in lis:
#   News Url
    key = li.find("span", {"class":"tit"})
    news_page_url = "https://search.naver.com/search.naver?where=news&sm=tab_jum&query=" + key.text
    req = requests.get(news_page_url, headers=headers)
    html = req.text
    soup = BeautifulSoup(html, "lxml")
    
    news_list = soup.find("ul", {"class":"type01"}).findAll('li')[0]
    source = news_list.find("span", {"class":"_sp_each_source"})
    link = news_list.find("a")["href"]
    
    req = requests.get(link, headers=headers)
    html = req.text

    a = Article(link, language="ko")
    a.download()
    a.parse()

    title = a.title
    content = a.text
    image = a.top_image
    company = source.text
    keyword = key.text
    date = a.publish_date
    
    print(content)
    
    cursor.execute("INSERT INTO article (title, content, image, company, keyword, date) VALUES(%s, %s, %s, %s, %s, %s)", (title, content, image, company, keyword, date))
    db.commit()
db.close()

김하균 기자 lama@asiae.co.kr









[아시아경제 김하균 기자] 기상청에 따르면 16일 중부지방은 대체로 맑다가 오후부터 구름이 많아지겠다. 강원 영동과 남부 지방에는 동풍의 영향으로 흐리고, 새벽에 남해안에서 비가 시작돼 낮에는 강원 영동과 경상 동해안으로 확대될 것으로 보인다.제주도는 제18호 태풍 '탈림'의 영향권에 들어 이날 종일 비가 내릴 것으로 예보됐다. 태풍 '탈림'은 이날 오후 3시쯤 서귀포 남쪽 약 330km 부근 해상에 위치할 것으로 예상된다.태풍의 영향으로 제주도는 이날 태풍특보가 발표될 가능성이 있고, 경상동해안과 제주도산지에도 많은 비가 내릴 것으로 보여 침수 등 피해에 각별한 주의가 요구된다.예상 강수량(16~17일)은 △경상동해안(15일~)50~100mm △제주도산지(15일~) 50~150mm 이상 △산지를 제외한 제주도(15일~) 30~80mm △강원영동·경북북동산간·경남남해안·울릉·독도 30~80mm △강원산지 20~100mm 이상 △강원영서·경상내륙·전남남해안 5~30mm 등이다.평년과 비슷한 기온이 예상되는 가운데 낮과 밤의 기온차는 클 것으로 보인다.아침 예상 최저기온은 △서울 17도 △대전 17도 △대구 17도 △전주 16도 △광주 17도 △부산 20도 △강릉 16도 △제주 21도 △울릉·독도 18도, 낮 예상 최고 기온은 △서울 27도 △대전 26도 △대구 25도 △전주 26도 △광주 27도 △부산 25도 △강릉 24도 △제주 25도 △울릉·독도 22도 등이다.이날은 태풍이 북상하면서 이미 풍랑주의보, 강풍주의보 등 기상특보가 내려진 제주 전 해상과 서해 남부 먼바다, 남해 먼바다 외에 동해상에도 바람이 매우 강하게 불고, 물결이 높게 일겠다.
모게리니 외교·안보 고위대표 성명 발표



(브뤼셀=연합뉴스) 김병수 특파원 = 페데리카 모게리니 유럽연합(EU) 외교·안보 고위대표는 15일 북한이 또다시 일본 상공을 지나가는 탄도미사일을 발사한 데 대해 "또 하나의 충격적인 도발"이라면서 북한에 대한 제